In [1]:
import geemap
import ee

In [2]:
ee.Initialize(project='useful-theory-442820-q8')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [3]:
# Create an interactive map
Map = geemap.Map()

# Access the Satellite Embedding Dataset
embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')


# Select a region
# ****************************************************

# Use the satellite basemap
Map.setOptions('SATELLITE')

# Define a polygon to specify the region of interest.
# You can replace these coordinates with your own.
geometry = ee.Geometry.Rectangle([
        [
            72.9201,
            5.628
        ],
        [
            72.9259,
            5.6325
        ]
    ])

# Center the map on the defined geometry
Map.centerObject(geometry, 14)

# Prepare the Satellite Embedding dataset
# ****************************************************

year = 2024
startDate = ee.Date.fromYMD(year, 1, 1)
endDate = startDate.advance(1, 'year')

filteredEmbeddings = embeddings.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(geometry))

embeddingsImage = filteredEmbeddings.mosaic()

#print('Satellite Embeddings Image', embeddingsImage)

# Visualize the Satellite Embedding Dataset
# *********************************************************

# Define visualization parameters to display three axes of the embedding space as an RGB image.
visParams = {'min': -0.3, 'max': 0.3, 'bands': ['A01', 'A16', 'A09']}

# Add the clipped embeddings image to the map.
Map.addLayer(embeddingsImage.clip(geometry), visParams, 'Embeddings Image')

# Create a training dataset for unsupervised clustering
# *********************************************************

nSamples = 1000
training = embeddingsImage.sample(
  region=geometry,
  scale=10,
  numPixels=nSamples,
  seed=100
)

# Print the first element of the training dataset to inspect it.
#print(training.first().getInfo())


# Function to train a model for the desired number of clusters
def getClusters(nClusters):
  """Trains a k-means clusterer and applies it to the image."""
  clusterer = ee.Clusterer.wekaKMeans(nClusters).train(training)

  # Cluster the image
  clustered = embeddingsImage.cluster(clusterer)
  return clustered


# Get and display clusters
# *********************************************************

# Get a clustered image with 2 clusters.
cluster2 = getClusters(2)
# Add the 2-cluster result to the map with a random visualizer.
Map.addLayer(cluster2.randomVisualizer().clip(geometry), {}, '2 clusters')




# Display the map
Map

Map(center=[5.63025000425882, 72.92299999997654], controls=(WidgetControl(options=['position', 'transparent_bg…